In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [26]:
maxTweets = 1000
attributes = ['id', 'date', 'rawContent', 'sourceLabel', 'user']

tweet_list = []

# can use from: since: lang: until:
for i, tweet in enumerate(sntwitter.TwitterSearchScraper('Milk lang:en since:2020-01-01 until:now').get_items()):

    if i > maxTweets:
        break

    tweet_list.append([tweet.__dict__[a] if a!="user" else tweet.__dict__[a].username \
                       for a in attributes ])

tweet_df = pd.DataFrame(tweet_list, columns=attributes)

In [27]:
tweet_df

,id,date,rawContent,sourceLabel,user
0,1568161361485778950,2022-09-09 08:56:18+00:00,now i knew why i’m not tall and had a low bmi ...,Twitter for iPhone,77lbss
1,1567919657700532229,2022-09-08 16:55:51+00:00,@georgemarshhh Not a 4th time tho lad not even...,Twitter for Android,Mark37941897
2,1567905608350814209,2022-09-08 16:00:01+00:00,I saw my barista making this with dairy milk. ...,Twitter for iPhone,haleyvemealone
3,1567873106978705415,2022-09-08 13:50:53+00:00,I’ve been feeling so sick in this heat…I feel ...,Twitter for iPhone,psValSays
4,1567858990566019077,2022-09-08 12:54:47+00:00,@BashirHashiysf You better take a break for li...,Twitter for Android,HenockYonas
...,...,...,...,...,...
996,1413071853090975753,2021-07-08 09:45:58+00:00,“How did you take care of it until now?” Chany...,Twitter for iPhone,nsfwcb1
997,1412826587372871686,2021-07-07 17:31:22+00:00,@bfs2020 I've just switched from milk chocolat...,Twitter Web App,trumpsatwit
998,1412823387165110275,2021-07-07 17:18:39+00:00,@SEC_News I aways felt unsure about milk up un...,Twitter for Android,HuguinhoT88
999,1412794178954801152,2021-07-07 15:22:35+00:00,@RemainerVermin @milk_and_more Shocking. I won...,Twitter for Android,shoestring230
